In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

import time
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder


from sklearn.tree import DecisionTreeClassifier

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

from math import sqrt
from pprint import pprint
from datetime import datetime
from itertools import cycle
from scipy import stats
import ipaddress
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import emlearn

In [2]:
df = pd.read_csv(f'artigo.csv', sep=';')
df['IAT_ms'] = df['IAT_ms'].fillna(0)
df

,TEID,IAT_ms,SRC_IP,SRC_PORT,DST_IP,DST_PORT,PKT_SIZE,PROTO,TCP_FLAGS,WIN_SIZE,SEQ,ACK_SEQ,RES1,DOFF,CHECKSUM,URG_PTR,LEN_UDP
0,1,0.000,10.60.0.1,44741,192.168.40.20,5006,60,6,2,65280,1753642859,0,0,10,23975,0,0
1,1,0.000,10.60.0.1,48663,192.168.40.20,5003,60,6,2,65280,3523827063,0,0,10,1737,0,0
2,1,0.000,10.60.0.1,60061,192.168.40.20,5002,60,6,2,65280,2256943670,0,0,10,16648,0,0
3,1,0.000,10.60.0.1,57247,192.168.40.20,5004,60,6,2,65280,2829296403,0,0,10,49417,0,0
4,1,0.000,10.60.0.1,51927,192.168.40.20,5005,60,6,2,65280,2289218262,0,0,10,58941,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4188080,1,7.502,10.60.0.1,51927,192.168.40.20,5005,52,6,16,510,2289218423,176599053,0,8,62578,0,0
4188081,1,16.143,10.60.0.1,45125,192.168.40.20,5026,52,6,16,510,1970129898,398439692,0,8,63305,0,0
4188082,1,16.118,10.60.0.1,55679,192.168.40.20,5003,52,6,16,510,1403731989,3066134647,0,8,4433,0,0
4188083,1,13.597,10.60.0.1,48663,192.168.40.20,5003,52,6,16,510,3523827224,4121944640,0,8,63391,0,0


In [3]:
aplicacao = [(df['SRC_PORT'] == 80)]
valor = ['eMBB']
df['APP'] = np.select(aplicacao, valor, default='URLLC')
df

,TEID,IAT_ms,SRC_IP,SRC_PORT,DST_IP,DST_PORT,PKT_SIZE,PROTO,TCP_FLAGS,WIN_SIZE,SEQ,ACK_SEQ,RES1,DOFF,CHECKSUM,URG_PTR,LEN_UDP,APP
0,1,0.000,10.60.0.1,44741,192.168.40.20,5006,60,6,2,65280,1753642859,0,0,10,23975,0,0,URLLC
1,1,0.000,10.60.0.1,48663,192.168.40.20,5003,60,6,2,65280,3523827063,0,0,10,1737,0,0,URLLC
2,1,0.000,10.60.0.1,60061,192.168.40.20,5002,60,6,2,65280,2256943670,0,0,10,16648,0,0,URLLC
3,1,0.000,10.60.0.1,57247,192.168.40.20,5004,60,6,2,65280,2829296403,0,0,10,49417,0,0,URLLC
4,1,0.000,10.60.0.1,51927,192.168.40.20,5005,60,6,2,65280,2289218262,0,0,10,58941,0,0,URLLC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4188080,1,7.502,10.60.0.1,51927,192.168.40.20,5005,52,6,16,510,2289218423,176599053,0,8,62578,0,0,URLLC
4188081,1,16.143,10.60.0.1,45125,192.168.40.20,5026,52,6,16,510,1970129898,398439692,0,8,63305,0,0,URLLC
4188082,1,16.118,10.60.0.1,55679,192.168.40.20,5003,52,6,16,510,1403731989,3066134647,0,8,4433,0,0,URLLC
4188083,1,13.597,10.60.0.1,48663,192.168.40.20,5003,52,6,16,510,3523827224,4121944640,0,8,63391,0,0,URLLC


In [4]:
df.to_csv('truetable.csv', sep=';')


In [5]:
fluxo = ['SRC_IP', 'DST_IP', 'SRC_PORT', 'DST_PORT', 'PROTO']

In [6]:
df['tempo_acumulado'] = df.groupby(fluxo)['IAT_ms'].cumsum()
df['timestamp_fluxo'] = pd.to_datetime(df['tempo_acumulado'], unit='ms')
df.head(10)

,TEID,IAT_ms,SRC_IP,SRC_PORT,DST_IP,DST_PORT,PKT_SIZE,PROTO,TCP_FLAGS,WIN_SIZE,SEQ,ACK_SEQ,RES1,DOFF,CHECKSUM,URG_PTR,LEN_UDP,APP,tempo_acumulado,timestamp_fluxo
0,1,0.000,10.60.0.1,44741,192.168.40.20,5006,60,6,2,65280,1753642859,0,0,10,23975,0,0,URLLC,0.000,1970-01-01 00:00:00.000000
1,1,0.000,10.60.0.1,48663,192.168.40.20,5003,60,6,2,65280,3523827063,0,0,10,1737,0,0,URLLC,0.000,1970-01-01 00:00:00.000000
2,1,0.000,10.60.0.1,60061,192.168.40.20,5002,60,6,2,65280,2256943670,0,0,10,16648,0,0,URLLC,0.000,1970-01-01 00:00:00.000000
3,1,0.000,10.60.0.1,57247,192.168.40.20,5004,60,6,2,65280,2829296403,0,0,10,49417,0,0,URLLC,0.000,1970-01-01 00:00:00.000000
4,1,0.000,10.60.0.1,51927,192.168.40.20,5005,60,6,2,65280,2289218262,0,0,10,58941,0,0,URLLC,0.000,1970-01-01 00:00:00.000000
5,1,0.000,10.60.0.1,41083,192.168.40.20,5001,60,6,2,65280,3606672532,0,0,10,88,0,0,URLLC,0.000,1970-01-01 00:00:00.000000
6,1,0.000,10.60.0.1,52189,192.168.40.20,5009,60,6,2,65280,1544540203,0,0,10,62527,0,0,URLLC,0.000,1970-01-01 00:00:00.000000
7,1,4.712,10.60.0.1,44741,192.168.40.20,5006,52,6,16,510,1753642860,342452082,0,8,28164,0,0,URLLC,4.712,1970-01-01 00:00:00.004712
8,1,4.696,10.60.0.1,48663,192.168.40.20,5003,52,6,16,510,3523827064,4121944635,0,8,47893,0,0,URLLC,4.696,1970-01-01 00:00:00.004696
9,1,4.697,10.60.0.1,60061,192.168.40.20,5002,52,6,16,510,2256943671,1136506713,0,8,53801,0,0,URLLC,4.697,1970-01-01 00:00:00.004697


In [7]:
janela = '500ms'
df_timeIndex = df.set_index('timestamp_fluxo')
agrupado = df_timeIndex.groupby(fluxo + ['APP'])
df_janela = agrupado['IAT_ms'].resample(janela).agg(['mean', 'var'])
df_janela = df_janela.dropna(subset=['mean'])
df_janela['var'] = df_janela['var'].fillna(0)
df_janela = df_janela.reset_index()
df_janela = df_janela.rename(columns={
    'mean':'media_iat',
    'var':'variancia_iat'
})
df_janela

,SRC_IP,DST_IP,SRC_PORT,DST_PORT,PROTO,APP,timestamp_fluxo,media_iat,variancia_iat
0,10.60.0.1,192.168.40.20,80,50646,6,eMBB,1970-01-01 00:00:00.000,3.317333,5.854926e+01
1,10.60.0.1,192.168.40.20,80,50646,6,eMBB,1970-01-01 00:00:30.000,30019.298000,0.000000e+00
2,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:00.000,0.477439,1.436323e+01
3,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:00.500,291.267000,0.000000e+00
4,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:01.000,515.428000,0.000000e+00
...,...,...,...,...,...,...,...,...,...
100694,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:14.500,5.918388,2.322013e+00
100695,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:15.000,8.243909,3.651326e+01
100696,10.60.0.1,192.168.40.20,60935,5021,6,URLLC,1970-01-01 00:00:00.000,11.992667,3.286768e+02
100697,10.60.0.1,192.168.40.20,60935,5021,6,URLLC,1970-01-01 00:16:15.000,325013.870000,3.168956e+11


In [8]:
#historico janelas anteriores
df_janela = df_janela.sort_values(by=fluxo + ['timestamp_fluxo'])
grupo_jan_anterior = df_janela.groupby(fluxo + ['APP'])
df_janela['media_jan_ant'] = grupo_jan_anterior['media_iat'].shift(1)
df_janela['variancia_jan_ant'] = grupo_jan_anterior['variancia_iat'].shift(1)
df_janela = df_janela.dropna().copy()
df_janela

,SRC_IP,DST_IP,SRC_PORT,DST_PORT,PROTO,APP,timestamp_fluxo,media_iat,variancia_iat,media_jan_ant,variancia_jan_ant
1,10.60.0.1,192.168.40.20,80,50646,6,eMBB,1970-01-01 00:00:30.000,30019.298000,0.000000e+00,3.317333,58.549260
3,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:00.500,291.267000,0.000000e+00,0.477439,14.363228
4,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:01.000,515.428000,0.000000e+00,291.267000,0.000000
5,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:01.500,32.191103,1.236399e+04,515.428000,0.000000
7,10.60.0.1,192.168.40.20,80,50660,6,eMBB,1970-01-01 00:00:00.500,521.392000,0.000000e+00,4.746458,735.465540
...,...,...,...,...,...,...,...,...,...,...,...
100692,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:13.500,7.072529,8.391878e+00,6.662895,6.079983
100693,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:14.000,6.924373,1.504854e+01,7.072529,8.391878
100694,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:14.500,5.918388,2.322013e+00,6.924373,15.048536
100695,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:15.000,8.243909,3.651326e+01,5.918388,2.322013


In [ ]:
df_janela.to_csv('500mstruetable.csv', sep=';')


In [10]:
counts = df_janela['APP'].value_counts()
perc = df_janela['APP'].value_counts(normalize=True) * 100
summa = pd.DataFrame({
    'Count': counts,
    '%': perc.round(2)
})
summa

,Count,%
APP,,
URLLC,58530,58.28
eMBB,41898,41.72


In [11]:
df_janela.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100428 entries, 1 to 100697
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   SRC_IP             100428 non-null  object        
 1   DST_IP             100428 non-null  object        
 2   SRC_PORT           100428 non-null  int64         
 3   DST_PORT           100428 non-null  int64         
 4   PROTO              100428 non-null  int64         
 5   APP                100428 non-null  object        
 6   timestamp_fluxo    100428 non-null  datetime64[ns]
 7   media_iat          100428 non-null  float64       
 8   variancia_iat      100428 non-null  float64       
 9   media_jan_ant      100428 non-null  float64       
 10  variancia_jan_ant  100428 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 9.2+ MB


In [ ]:
mapa_apps = {'URLLC': 0, 'eMBB': 1}

df_janela['APP_NUM'] = df_janela['APP'].map(mapa_apps)

df_janela['APP_NUM'] = df_janela['APP_NUM'].fillna(-1).astype(int)
df_janela

,SRC_IP,DST_IP,SRC_PORT,DST_PORT,PROTO,APP,timestamp_fluxo,media_iat,variancia_iat,media_jan_ant,variancia_jan_ant,APP_NUM
1,10.60.0.1,192.168.40.20,80,50646,6,eMBB,1970-01-01 00:00:30.000,30019.298000,0.000000e+00,3.317333,58.549260,1
3,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:00.500,291.267000,0.000000e+00,0.477439,14.363228,1
4,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:01.000,515.428000,0.000000e+00,291.267000,0.000000,1
5,10.60.0.1,192.168.40.20,80,50658,6,eMBB,1970-01-01 00:00:01.500,32.191103,1.236399e+04,515.428000,0.000000,1
7,10.60.0.1,192.168.40.20,80,50660,6,eMBB,1970-01-01 00:00:00.500,521.392000,0.000000e+00,4.746458,735.465540,1
...,...,...,...,...,...,...,...,...,...,...,...,...
100692,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:13.500,7.072529,8.391878e+00,6.662895,6.079983,0
100693,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:14.000,6.924373,1.504854e+01,7.072529,8.391878,0
100694,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:14.500,5.918388,2.322013e+00,6.924373,15.048536,0
100695,10.60.0.1,192.168.40.20,60789,5016,6,URLLC,1970-01-01 00:16:15.000,8.243909,3.651326e+01,5.918388,2.322013,0


In [13]:
counts = df_janela['APP_NUM'].value_counts()
perc = df_janela['APP_NUM'].value_counts(normalize=True) * 100
summa = pd.DataFrame({
    'Count': counts,
    '%': perc.round(2)
})
summa

,Count,%
APP_NUM,,
0,58530,58.28
1,41898,41.72


In [14]:
counts = df_janela['APP'].value_counts()
perc = df_janela['APP'].value_counts(normalize=True) * 100
summa = pd.DataFrame({
    'Count': counts,
    '%': perc.round(2)
})
summa

,Count,%
APP,,
URLLC,58530,58.28
eMBB,41898,41.72


In [15]:
X_SEN = df_janela.drop(columns=["DST_IP", "SRC_IP", 'SRC_PORT', 'DST_PORT', 'APP', 'APP_NUM', 'PROTO', 'timestamp_fluxo']).copy()
y_SEN = df_janela ['APP_NUM'].copy()

In [16]:
X_train_SEN, X_val_SEN, y_train_SEN, y_val_SEN = train_test_split(X_SEN, y_SEN, test_size=0.20, random_state=42, shuffle=True)

In [17]:
model = RandomForestClassifier(
    n_estimators=10,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42
)

In [18]:
model.fit(X_train_SEN, y_train_SEN)

,n_estimators,10
,criterion,'gini'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [19]:
y_pred = model.predict(X_val_SEN)

In [22]:
cmodel_c = emlearn.convert(model, method='inline', dtype='int64_t')
cmodel_c.save(file='model500msJANELA.h', name='model')
cmodel_c

In [20]:
print("Accuracy:", accuracy_score(y_val_SEN, y_pred)) 
print("Precision:", precision_score(y_val_SEN, y_pred, average='weighted')) 
print("Recall:", recall_score(y_val_SEN, y_pred, average='weighted')) 
print("F1 Score:", f1_score(y_val_SEN, y_pred, average='weighted'))

Accuracy: 0.99970128447675
Precision: 0.999701384645794
Recall: 0.99970128447675
F1 Score: 0.99970129393913
